# Import Statements

In [ ]:
# Import packages 
import pandas as pd
import numpy as np
import netCDF4
import h5netcdf
import xarray as xr
from os.path import join
import joblib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import shapely
import cartopy
%matplotlib inline
import datetime as dt

import sys
sys.path.append('/home/samuel.varga/projects/2to6_hr_severe_wx/')
sys.path.append('/home/samuel.varga/python_packages/ml_workflow/')
sys.path.append('/home/samuel.varga/python_packages/MontePython/')
sys.path.append('/home/samuel.varga/python_packages/WoF_post') #WoF post package
sys.path.append('/home/samuel.varga/python_packages/wofs_ml_severe/')
sys.path.append('/home/samuel.varga/python_packages/VargaPy/')
from main.io import load_ml_data

from VargaPy.MlUtils import Drop_Unwanted_Variables, get_bl_col, pseudo_all_severe_probs
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
from wofs.plotting.wofs_colors import WoFSColors
from wofs_ml_severe.data_pipeline.storm_report_loader import StormReportLoader
from wofs_ml_severe.data_pipeline.storm_report_downloader import StormReportDownloader

# Load Files

# 3 km Base 

# 9 km Upscaled

# 9 km Time Composite

# 9 km Neighborhoods & Statistics

# Target Visualization

# 3 km: Target Mapping

# 3 km: 9km Filter

# 9km: Targets on Upscaled Grid

In [ ]:
def get_targets(target_scale, base_path, date, bl_column, FRAMEWORK=FRAMEWORK, TIMESCALE=TIMESCALE, append_UH=False, Three_km=False, return_targets=True, mode=None, SigSevere=SigSev, appendUH=False, Tkm=False): #Return targets for a given day to produce heatmap
        if SigSevere:
            target_col=f'wind_sig_severe__{target_scale}km'
        else:
            target_col=f'wind_severe__{target_scale}km'
        X, X_bl, y = load_ml_data(base_path=base_path, date=date,
                                      mode=mode,
                                      target_col=target_col,
                                      bl_column=bl_column,
                                      FRAMEWORK=FRAMEWORK,
                                      TIMESCALE=TIMESCALE, appendUH=appendUH, Three_km=Three_km,
                                      return_targets=return_targets)
        print(len(y[y>0])) #Number of points with wind targets
        for hazard in ['hail','tornado']:
            if SigSevere:
                target_col='{}_sig_severe__{}km'.format(hazard, target_scale)
            else:
                target_col='{}_severe__{}km'.format(hazard, target_scale)
            _, _, y1  = load_ml_data(base_path=base_path, date=date,
                                           mode=mode,
                                           target_col=target_col,
                                           bl_column=bl_column,
                                           FRAMEWORK=FRAMEWORK,
                                           TIMESCALE=TIMESCALE, Three_km=Three_km,
                                    return_targets=return_targets) 
            y +=y1
            print(len(y[y>0])) #Prints the number of wind+hail targets, then wind+hail+tornado

        y[y > 0] = 1 #All target points (y>1) are remapped to y=1
    
        return y 
    
def get_pseudo_all_severe_probs(target_scale, base_path=base_path, date=date, bl_column=bl_column, shape=shape): 
    targets=np.array(get_targets(target_scale,base_path, date, bl_column))
    targets_2D=targets.reshape(shape)
    return targets_2D

In [ ]:
%matplotlib inline

fig, axes = plt.subplots(figsize=(10,8), facecolor='w',
                     dpi=170, nrows=1, ncols=2, subplot_kw={'projection': projection}, 
                        constrained_layout=True)

    
for ax, target_scale in zip(axes.flat, ['36','9']):
    targets_2D=get_pseudo_all_severe_probs(target_scale)
#36km and 9km scales  
    add_map_stuff(ax, states, shape_feature)
    targets_2D = np.ma.masked_where(targets_2D==0, targets_2D)

        #cmap=WoFSColors.wz_cmap
    cf = ax.contourf(lons,lats, targets_2D, cmap=WoFSColors.wz_cmap, alpha=0.95, levels=[0,1],
                    transform = crs, )

        ##point_lat, point_lon = points[hazard]
    if hazard=='all':
            for Lorem in ['hail','wind','tornado']:
                _points=points[Lorem]
                axes[0].scatter(_points[:,1],_points[:,0], s=10, color=hazard_color[Lorem], alpha=0.8, zorder=1, transform=crs)
    else:
            _points = points[hazard]
            ax.scatter(_points[:,1],_points[:,0], s=900, color='k', alpha=0.8, zorder=1, transform=crs) #s=10


    ax = set_extent(ax, projection , crs, lats, lons,)
    fontsize = 10 if len(name) > 5 else 12
    ax.set_title(f'Regions within {target_scale}km of a Storm Report', fontsize=fontsize)

    fig.tight_layout()
    #Add 36/18/9km radius to storm reports
    if False:  
        if hazard=='all':
                for Lorem in ['hail','wind','tornado']:
                    _points = points[Lorem]
                    for pair in zip(_points[::3,1], _points[::3,0]):
                        repX, repY = circleofRadius(pair[1],pair[0], 36)
                        axes.plot(repX, repY, transform=crs, color='k', linewidth=1, alpha=0.5)
        else:
                for pair in zip(_points[:,1], _points[:,0]):
                    repX, repY = circleofRadius(pair[1],pair[0], 36)
                    axes.plot(repX, repY, transform=crs, color='k', linewidth=1)
axes[1].annotate(title, (0.48, 1.07), xycoords='axes fraction', fontsize=10, color='k')
plt.show()